# White House visitor logs

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import datetime as dt

In [3]:
import altair as alt
import altair_latimes as lat
import numpy as np

In [4]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")

ThemeRegistry.enable('latimes')

In [5]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.options.display.max_colwidth = None

### Data dictionary

In [6]:
# "https://obamawhitehouse.archives.gov/files/disclosures/visitors/WhiteHouse-WAVES-Key-1209.txt"

In [7]:
today = dt.date.today().strftime("%m-%d-%Y")

In [8]:
df = pd.read_csv(
    "data/processed/log_latest.csv",
    parse_dates=[
        "appt_made_date",
        "appt_start_date",
        "appt_end_date",
        "appt_cancel_date",
        "releasedate",
    ],
)

In [9]:
df.sort_values(["total_people"], ascending=False).head()

,namelast,namefirst,namemid,uin,bdgnbr,access_type,toa,poa,tod,pod,appt_made_date,appt_start_date,appt_end_date,appt_cancel_date,total_people,last_updatedby,post,lastentrydate,terminal_suffix,visitee_namelast,visitee_namefirst,meeting_loc,meeting_room,caller_name_last,caller_name_first,caller_room,description,releasedate
3210,CORTEZ,TRISTEN,R,U23376,NaN,VA,4/16/2021 11:57,SOS,NAN,NAN,2021-04-10 14:16:00,2021-04-16,2021-04-16 23:59:00,NaT,95.0,ET,WIN,4/10/2021 14:16,ET,TELEKY,ED,WH,EW 216,TELEKY,EDWARD,NaN,NAN,2021-07-29
3197,BELLOT,ALEIA,T,U23376,NaN,VA,4/16/2021 12:04,SOS,NAN,NAN,2021-04-10 14:16:00,2021-04-16,2021-04-16 23:59:00,NaT,95.0,ET,WIN,4/10/2021 14:16,ET,TELEKY,ED,WH,EW 216,TELEKY,EDWARD,NaN,NAN,2021-07-29
3253,MAYS,OMARI,J,U23376,NaN,VA,4/16/2021 12:09,SOS,NAN,NAN,2021-04-10 14:16:00,2021-04-16,2021-04-16 23:59:00,NaT,95.0,ET,WIN,4/10/2021 14:16,ET,TELEKY,ED,WH,EW 216,TELEKY,EDWARD,NaN,NAN,2021-07-29
3271,ROY,KEVIN,M,U23376,NaN,VA,NAN,NAN,NAN,NAN,2021-04-12 09:56:00,2021-04-16,2021-04-16 23:59:00,NaT,95.0,ET,WIN,4/12/2021 9:56,ET,TELEKY,ED,WH,EW 216,TELEKY,EDWARD,NaN,NAN,2021-07-29
3272,SANTIAGO,ANDREW,B,U23376,NaN,VA,4/16/2021 11:54,SOS,NAN,NAN,2021-04-12 09:56:00,2021-04-16,2021-04-16 23:59:00,NaT,95.0,ET,WIN,4/12/2021 9:56,ET,TELEKY,ED,WH,EW 216,TELEKY,EDWARD,NaN,NAN,2021-07-29


### Buildings

In [10]:
df.meeting_loc.value_counts()

OEOB    1579
WH      1398
EEOB     837
NEOB     374
Name: meeting_loc, dtype: int64

### Rooms 

In [11]:
df.meeting_room.value_counts()

234           316
EW206         231
472           209
97            137
EEOB 210/2    125
3201          120
445           111
OVAL OFFIC    108
STATE PLAC    108
ROOSEVELT      99
430            97
EW 216         97
RES - STAT     92
374            77
3202           72
EAST ROOM      62
415            51
210            49
WW             45
RES CLINIC     43
VP CEREMON     40
ROSE GARDE     40
183            33
94             33
76             32
291            31
386            30
312            28
SOUTH COUR     24
SITUATION      22
BR             22
21             22
353            21
367            21
1              21
WHSR           21
WEST EXEC      20
206            20
276            19
5020           19
474            19
393            18
419            18
STATE PL.      18
311            17
490            17
WW/ROSE GA     17
4000           16
425            16
27             16
265            16
18             16
450            16
5013           16
422            16
PASS OFFIC

### Who's visited the Oval?

In [12]:
len(df[df["meeting_room"].str.contains("OVAL OFFIC")])

108

In [13]:
df[df["meeting_room"].str.contains("OVAL OFFIC")].head()

,namelast,namefirst,namemid,uin,bdgnbr,access_type,toa,poa,tod,pod,appt_made_date,appt_start_date,appt_end_date,appt_cancel_date,total_people,last_updatedby,post,lastentrydate,terminal_suffix,visitee_namelast,visitee_namefirst,meeting_loc,meeting_room,caller_name_last,caller_name_first,caller_room,description,releasedate
115,ESHOO,ANNA,G,U21725,NaN,VA,3/3/21 13:31,B0401,NAN,NAN,2021-03-03 11:27:00,2021-03-03 13:30:00,2021-03-03 23:59:00,NaT,9.0,GT,WIN,3/3/2021,GT,NAN,POTUS,WH,OVAL OFFIC,TAYLOR,GABRIELLE,NaN,NAN,2021-06-29
116,VANHOLLEN,CHRISTOPHER,N,U21725,NaN,VA,3/3/21 13:27,B0401,NAN,NAN,2021-03-03 11:27:00,2021-03-03 13:30:00,2021-03-03 23:59:00,NaT,9.0,GT,WIN,3/3/2021,GT,NAN,POTUS,WH,OVAL OFFIC,TAYLOR,GABRIELLE,NaN,NAN,2021-06-29
117,DEGETTE,DIANA,L,U21725,NaN,VA,NAN,NAN,NAN,NAN,2021-03-03 11:27:00,2021-03-03 13:30:00,2021-03-03 23:59:00,NaT,9.0,GT,WIN,3/3/2021,GT,NAN,POTUS,WH,OVAL OFFIC,TAYLOR,GABRIELLE,NaN,NAN,2021-06-29
118,UPTON,FREDERICK,S,U21725,NaN,VA,NAN,NAN,NAN,NAN,2021-03-03 11:27:00,2021-03-03 13:30:00,2021-03-03 23:59:00,NaT,9.0,GT,WIN,3/3/2021,GT,NAN,POTUS,WH,OVAL OFFIC,TAYLOR,GABRIELLE,NaN,NAN,2021-06-29
121,CRAPO,MICHAEL,D,U21725,NaN,VA,NAN,NAN,NAN,NAN,2021-03-03 11:27:00,2021-03-03 13:30:00,2021-03-03 23:59:00,NaT,9.0,GT,WIN,3/3/2021,GT,NAN,POTUS,WH,OVAL OFFIC,TAYLOR,GABRIELLE,NaN,NAN,2021-06-29


### Where meeting room is ...

In [14]:
df[df["meeting_room"].str.contains("EW206")].head()

,namelast,namefirst,namemid,uin,bdgnbr,access_type,toa,poa,tod,pod,appt_made_date,appt_start_date,appt_end_date,appt_cancel_date,total_people,last_updatedby,post,lastentrydate,terminal_suffix,visitee_namelast,visitee_namefirst,meeting_loc,meeting_room,caller_name_last,caller_name_first,caller_room,description,releasedate
288,MENDEZ,ANTHONY,A,U21895,NaN,VA,NAN,NAN,NAN,NAN,2021-03-08 08:41:00,2021-03-08 10:30:00,2021-03-08 23:59:00,NaT,5.0,DV,WIN,3/8/2021,DV,VIA,DAN,WH,EW206,VIA,DANIEL,NaN,NAN,2021-06-29
290,CAMPBELL,EVAN,M,U21895,NaN,VA,NAN,NAN,NAN,NAN,2021-03-08 08:41:00,2021-03-08 10:30:00,2021-03-08 23:59:00,NaT,5.0,DV,WIN,3/8/2021,DV,VIA,DAN,WH,EW206,VIA,DANIEL,NaN,NAN,2021-06-29
291,KLIMCZAK,FRANCES,D,U21895,NaN,VA,NAN,NAN,NAN,NAN,2021-03-08 08:41:00,2021-03-08 10:30:00,2021-03-08 23:59:00,NaT,5.0,DV,WIN,3/8/2021,DV,VIA,DAN,WH,EW206,VIA,DANIEL,NaN,NAN,2021-06-29
292,AHRENS,KATHERINE,L,U21895,NaN,VA,NAN,NAN,NAN,NAN,2021-03-08 08:41:00,2021-03-08 10:30:00,2021-03-08 23:59:00,NaT,5.0,DV,WIN,3/8/2021,DV,VIA,DAN,WH,EW206,VIA,DANIEL,NaN,NAN,2021-06-29
440,REGINA,ANTHONY,C,U22043,NaN,VA,3/11/21 13:02,SOS,NAN,NAN,2021-03-10 11:34:00,2021-03-11 13:00:00,2021-03-11 23:59:00,NaT,22.0,DV,WIN,3/10/2021,DV,VIA,DAN,WH,EW206,VIA,DANIEL,NaN,NAN,2021-06-29


### Where 'visitee' is explicity the Vice President

In [15]:
df[df["visitee_namefirst"].str.contains("VPOTUS")]

,namelast,namefirst,namemid,uin,bdgnbr,access_type,toa,poa,tod,pod,appt_made_date,appt_start_date,appt_end_date,appt_cancel_date,total_people,last_updatedby,post,lastentrydate,terminal_suffix,visitee_namelast,visitee_namefirst,meeting_loc,meeting_room,caller_name_last,caller_name_first,caller_room,description,releasedate
726,CORMACK,MAUREEN,E,U22244,177684.0,VA,3/17/21 9:34,D1101,3/17/2021,B04,2021-03-16 09:30:00,2021-03-17 09:00:00,2021-03-17 23:59:00,NaT,1.0,WD,WIN,3/16/2021,WD,NAN,VPOTUS,OEOB,291,DIRKSEN,WARD,NaN,NAN,2021-06-29
1099,ZUNIGA,RICARDO,F,U22762,NaN,VA,NAN,NAN,NAN,NAN,2021-03-27 11:12:00,2021-03-29 14:00:00,2021-03-29 23:59:00,NaT,1.0,GW,WIN,3/27/2021,GW,NAN,VPOTUS,WH,VP OFFICE,WOOLFOLK,GARLAND,NaN,NAN,2021-06-29
1100,ZUNIGA,RICARDO,F,U22793,178331.0,VA,3/29/21 15:00,B0401,NAN,NAN,2021-03-29 09:32:00,2021-03-29 16:00:00,2021-03-29 23:59:00,NaT,1.0,GW,WIN,3/29/2021,GW,NAN,VPOTUS,WH,WHSR,WOOLFOLK,GARLAND,NaN,NAN,2021-06-29
3439,AGUILAR,JOSE,A,U23887,176506.0,VA,4/22/2021 8:26,B0401,4/22/2021 11:47,A05,2021-04-21 10:57:00,2021-04-22 00:00:00,2021-04-22 23:59:00,NaT,1.0,GW,WIN,4/21/2021 10:57,GW,NAN,VPOTUS,OEOB,274,WOOLFOLK,GARLAND,NaN,NAN,2021-07-29
3441,HEINTZ,STEPHEN,N,U23885,177900.0,VA,4/22/2021 8:49,D1102,NAN,NAN,2021-04-21 10:38:00,2021-04-22 00:00:00,2021-04-22 23:59:00,NaT,1.0,GW,WIN,4/21/2021 10:38,GW,NAN,VPOTUS,OEOB,270,WOOLFOLK,GARLAND,NaN,NAN,2021-07-29
3454,MCINTYRE,NICOLA,N,U23883,178370.0,VA,4/22/2021 8:23,B0401,4/22/2021 11:47,A05,2021-04-21 10:12:00,2021-04-22 00:00:00,2021-04-22 23:59:00,NaT,1.0,GW,WIN,4/21/2021 10:12,GW,NAN,VPOTUS,OEOB,274,WOOLFOLK,GARLAND,NaN,NAN,2021-07-29
3455,OLSON,ERIC,L,U23858,178733.0,VA,4/22/2021 8:24,B0401,NAN,NAN,2021-04-21 07:10:00,2021-04-22 00:00:00,2021-04-22 23:59:00,NaT,1.0,GW,WIN,4/21/2021 7:10,GW,NAN,VPOTUS,OEOB,274,WOOLFOLK,GARLAND,NaN,NAN,2021-07-29
3456,WALKER,DARREN,C,U23918,177924.0,VA,4/22/2021 8:21,B0401,4/22/2021 11:41,B04,2021-04-21 16:14:00,2021-04-22 00:00:00,2021-04-22 23:59:00,NaT,1.0,GW,WIN,4/21/2021 16:14,GW,NAN,VPOTUS,OEOB,274,WOOLFOLK,GARLAND,NaN,NAN,2021-07-29
3753,ADACHI,MASASHI,N,U23610,NaN,VA,NAN,NAN,NAN,NAN,2021-04-15 15:43:00,2021-04-16 00:00:00,2021-04-16 23:59:00,NaT,6.0,WD,WIN,4/15/2021 15:43,WD,NAN,VPOTUS,OEOB,276,DIRKSEN,WARD,NaN,NAN,2021-07-29
3761,FUJINUMA,ATSUYUKI,N,U23610,NaN,VA,NAN,NAN,NAN,NAN,2021-04-16 06:22:00,2021-04-16 00:00:00,2021-04-16 23:59:00,NaT,6.0,WD,WIN,4/16/2021 6:22,WD,NAN,VPOTUS,OEOB,276,DIRKSEN,WARD,NaN,NAN,2021-07-29
